In [168]:
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, MT5ForConditionalGeneration, MT5Tokenizer
import pandas as pd
import numpy as np
import os
import xml.etree.ElementTree as ET
from sklearn.metrics import f1_score
from pathlib import Path
from pandarallel import pandarallel
from multiprocessing import Pool
import random

In [126]:
pathlist = Path('en-test').glob('**/*.xml')
ids=[]
x_raw=[]
for path in pathlist:  
    head, tail = os.path.split(path)
    t=tail.split(".")
    path_in_str = str(path)
    tree = ET.parse(path_in_str)
    root = tree.getroot()
    for child in root:
        xi=[]
        for ch in child:
            ids.append(t[0])
            x_raw.append(ch.text)

test_df=pd.DataFrame()
test_df["id"]=ids
test_df["tweets"]=x_raw

In [127]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20000 non-null  object
 1   tweets  20000 non-null  object
dtypes: object(2)
memory usage: 312.6+ KB


In [128]:
model_alt = T5ForConditionalGeneration.from_pretrained("PaulAdversarial/T5_PAN_Hate_Speech_Twitter_topic_author_ishatespeach")

In [129]:
model = T5ForConditionalGeneration.from_pretrained("PaulAdversarial/T5_PAN_Hate_Speech_Twitter_topic_ishatespeach")

In [130]:
tokenizer = T5Tokenizer.from_pretrained("PaulAdversarial/T5_PAN_Hate_Speech_Twitter_topic_author_ishatespeach")

In [131]:
df = pd.read_csv('train_df.csv')
#reading the pre-processed data

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [132]:
df.head()

,Unnamed: 0,input_text,prefix,target,output,output_alt
0,0,rt user funny how 15 days to slow the spread t...,hater classification,neutral,neutral,neutral
1,1,rt user why did minneapolis just give george f...,hater classification,neutral,neutral,neutral
2,2,rt user to be fair he has done a lot of underc...,hater classification,neutral,neutral,neutral
3,3,rt user president trump got us the hashtag vac...,hater classification,neutral,hatespeach,hatespeach
4,4,rt user is the case against former officer der...,hater classification,neutral,neutral,neutral


In [135]:
df=df.drop(columns="Unnamed: 0")

In [136]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160000 entries, 0 to 159999
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   input_text  160000 non-null  object
 1   prefix      160000 non-null  object
 2   target      160000 non-null  object
 3   output      10000 non-null   object
 4   output_alt  10000 non-null   object
dtypes: object(5)
memory usage: 6.1+ MB


In [137]:
tweet = str(df.iloc[80007,1])+ ": "+ df.iloc[80007,0]
print(tweet)
input_ids = tokenizer.encode(tweet, return_tensors = "pt")
output_ids = model.generate(input_ids)
print(tokenizer.decode(output_ids[0], skip_special_tokens=True))

topic classification: rt user happy friday beautiful people 
class: 78


In [138]:
df['output'] = ""
df['output_alt'] = ""
#df['input'] = df['input_text'].apply(lambda x: "hater classification: " + x)
#df['input_ids'] = ""
#df['output_ids'] = ""
#df['output_alt_ids'] = ""

In [140]:
# ok looks everythong works - let's predict and see the results on train
def parallelize_dataframe(df, func, n_cores=4):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df


def predict(df):
    df['input_ids'] = df['input'].apply(lambda x: tokenizer.encode(x, return_tensors = "pt"))
    df['output_ids'] = df['input_ids'].apply(lambda x: model.generate(x))
    df['output_alt_ids'] = df['input_ids'].apply(lambda x: model_alt.generate(x))
    df['output'] = df['output_ids'].apply(lambda x: str(tokenizer.decode(x, skip_special_tokens=True)))
    df['output_alt'] = df['output_alt_ids'].apply(lambda x: str(tokenizer.decode(x, skip_special_tokens=True)))
    return df

def straight_predict(df):
    for i in range(0,40000):
        #print(i)
        tweet = "hater classification: "+ df.iloc[i,0]
        input_ids = tokenizer.encode(tweet, return_tensors = "pt")
        output_ids = model.generate(input_ids)
        output_a_ids = model_alt.generate(input_ids)
        output = str(tokenizer.decode(output_ids[0], skip_special_tokens=True))
        output_a = str(tokenizer.decode(output_a_ids[0], skip_special_tokens=True))
        df.iloc[i,3]=output
        df.iloc[i,4]=output_a
        if i%4000==0:
            print("{} % complete".format(i/400))
    return df

In [141]:
#df = parallelize_dataframe(df, straight_predict, n_cores=20)
df = straight_predict(df)

0.0 % complete
10.0 % complete
20.0 % complete
30.0 % complete
40.0 % complete
50.0 % complete
60.0 % complete
70.0 % complete
80.0 % complete
90.0 % complete


In [142]:
def replace_bin(x):
    if x=='neutral':
        x=0
    elif(x=='hatespeach'):
        x=1
    else:
        print(x)
        x=0
    return int(x)

In [146]:
df = df[0:40000]

In [147]:
df.head()

,input_text,prefix,target,output,output_alt
0,rt user funny how 15 days to slow the spread t...,hater classification,neutral,neutral,neutral
1,rt user why did minneapolis just give george f...,hater classification,neutral,neutral,neutral
2,rt user to be fair he has done a lot of underc...,hater classification,neutral,neutral,neutral
3,rt user president trump got us the hashtag vac...,hater classification,neutral,hatespeach,hatespeach
4,rt user is the case against former officer der...,hater classification,neutral,neutral,neutral


In [148]:
df.output_alt.value_counts()

neutral       25783
hatespeach    14217
Name: output_alt, dtype: int64

In [149]:
df.target.value_counts()

neutral       20000
hatespeach    20000
Name: target, dtype: int64

In [150]:
#t5 generatoer did not ework with parallel computing due to some bugs so we start to using it only here
pandarallel.initialize()
df['target_num'] = df['target'].parallel_apply(replace_bin)
df['output_num'] = df['output'].parallel_apply(replace_bin)
df['output_alt_num'] = df['output_alt'].parallel_apply(replace_bin)

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
neutral: 104
0


In [151]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   input_text      40000 non-null  object
 1   prefix          40000 non-null  object
 2   target          40000 non-null  object
 3   output          40000 non-null  object
 4   output_alt      40000 non-null  object
 5   target_num      40000 non-null  int64 
 6   output_num      40000 non-null  int64 
 7   output_alt_num  40000 non-null  int64 
dtypes: int64(3), object(5)
memory usage: 2.4+ MB


In [152]:
def assign_max(df,step=200, weight=0.35):
    for i in range(0, int(df.shape[0]/step)):
        df.loc[i*step:(i+1)*step-1,'output_num_avg'] = int(round(df.loc[i*step:(i+1)*step-1,'output_num'].mean()))
        df.loc[i*step:(i+1)*step-1,'output_alt_num_avg'] = int(round(df.loc[i*step:(i+1)*step-1,'output_alt_num'].mean()))
        df.loc[i*step:(i+1)*step-1,'output_mix_num_avg'] = int(round(weight*df.loc[i*step:(i+1)*step-1,'output_alt_num'].mean()+(1-weight)*df.loc[i*step:(i+1)*step-1,'output_num'].mean()))
    return df

In [153]:
df = assign_max(df)

In [154]:
df.output_num_avg = df.output_num_avg.astype(int)
df.output_alt_num_avg = df.output_alt_num_avg.astype(int)
df.output_mix_num_avg = df.output_mix_num_avg.astype(int)

In [155]:
df.output_num.describe()

count    40000.000000
mean         0.578425
std          0.493817
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: output_num, dtype: float64

In [156]:
df.target_num.describe()

count    40000.000000
mean         0.500000
std          0.500006
min          0.000000
25%          0.000000
50%          0.500000
75%          1.000000
max          1.000000
Name: target_num, dtype: float64

In [157]:
df.describe()

,target_num,output_num,output_alt_num,output_num_avg,output_alt_num_avg,output_mix_num_avg
count,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000
mean,0.500000,0.578425,0.355425,0.585000,0.355000,0.500000
std,0.500006,0.493817,0.478648,0.492728,0.478519,0.500006
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.500000,1.000000,0.000000,1.000000,0.000000,0.500000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [158]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   input_text          40000 non-null  object
 1   prefix              40000 non-null  object
 2   target              40000 non-null  object
 3   output              40000 non-null  object
 4   output_alt          40000 non-null  object
 5   target_num          40000 non-null  int64 
 6   output_num          40000 non-null  int64 
 7   output_alt_num      40000 non-null  int64 
 8   output_num_avg      40000 non-null  int64 
 9   output_alt_num_avg  40000 non-null  int64 
 10  output_mix_num_avg  40000 non-null  int64 
dtypes: int64(6), object(5)
memory usage: 3.4+ MB


In [159]:
df.head(15)

,input_text,prefix,target,output,output_alt,target_num,output_num,output_alt_num,output_num_avg,output_alt_num_avg,output_mix_num_avg
0,rt user funny how 15 days to slow the spread t...,hater classification,neutral,neutral,neutral,0,0,0,0,0,0
1,rt user why did minneapolis just give george f...,hater classification,neutral,neutral,neutral,0,0,0,0,0,0
2,rt user to be fair he has done a lot of underc...,hater classification,neutral,neutral,neutral,0,0,0,0,0,0
3,rt user president trump got us the hashtag vac...,hater classification,neutral,hatespeach,hatespeach,0,1,1,0,0,0
4,rt user is the case against former officer der...,hater classification,neutral,neutral,neutral,0,0,0,0,0,0
5,rt user the fbi cia and military now feel tota...,hater classification,neutral,hatespeach,neutral,0,1,0,0,0,0
6,rt user biden bashed for taking all credit for...,hater classification,neutral,neutral,hatespeach,0,0,1,0,0,0
7,rt user biden's disappointing prime time spee...,hater classification,neutral,hatespeach,neutral,0,1,0,0,0,0
8,rt user cuomo should not be worried about resi...,hater classification,neutral,neutral,neutral,0,0,0,0,0,0
9,rt user breaking family of george floyd settle...,hater classification,neutral,hatespeach,neutral,0,1,0,0,0,0


In [160]:
f1_score(df['target_num'], df['output_num'])

0.7613417715650138

In [161]:
f1_score(df['target_num'], df['output_num_avg'])

0.9216589861751151

In [162]:
f1_score(df['target_num'], df['output_alt_num_avg'])

0.8304093567251462

In [163]:
f1_score(df['target_num'], df['output_mix_num_avg'])

0.99

In [164]:
#ok looks good - now lets see what is with the spanish one
dfs = pd.read_csv("train_es_df.csv")


In [165]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160000 entries, 0 to 159999
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  160000 non-null  int64 
 1   index       160000 non-null  int64 
 2   input_text  160000 non-null  object
 3   prefix      160000 non-null  object
 4   target      160000 non-null  object
dtypes: int64(2), object(3)
memory usage: 6.1+ MB


In [172]:
dfs.head()

,input_text,prefix,target
0,rt user esperando para ver la captura de jaime...,hater classification,neutral
1,user user que bueno romeo con la ayuda de dios...,hater classification,neutral
2,rt user nuestras cuadrillas están restaurando ...,hater classification,neutral
3,rt user esta es la portada de hashtag de este ...,hater classification,neutral
4,rt user lo más matado de la risa que vas a ver...,hater classification,neutral


In [171]:
dfs = dfs.drop(columns = ['Unnamed: 0','index'])

In [169]:
multi_model = MT5ForConditionalGeneration.from_pretrained("outputs/model_files")
multi_tokenizer = MT5Tokenizer.from_pretrained("outputs/model_files")

In [170]:
dfs = dfs[0:40000]

In [175]:
dfs['output'] = ""

In [176]:
def full_predict(df, start, stop, step, model, tokenizer):
    for i in range(start, stop):
        tweet = "hater classification: "+ df.iloc[i,0]
        input_ids = tokenizer.encode(tweet, return_tensors = "pt")
        output_ids = model.generate(input_ids)
        output = str(tokenizer.decode(output_ids[0], skip_special_tokens=True))
        df.iloc[i,3]=output
        if i%step==0:
            print("{} % complete".format(i/(step/10)))
    return df

In [177]:
dfs = full_predict(dfs, 0, 40000, 4000, multi_model, multi_tokenizer)

0.0 % complete
0.1 % complete
0.2 % complete
0.3 % complete
0.4 % complete
0.5 % complete
0.6 % complete
0.7 % complete
0.8 % complete
0.9 % complete


In [178]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   input_text  40000 non-null  object
 1   prefix      40000 non-null  object
 2   target      40000 non-null  object
 3   output      40000 non-null  object
dtypes: object(4)
memory usage: 1.2+ MB


In [179]:
dfs['target_num'] = dfs['target'].parallel_apply(replace_bin)
dfs['output_num'] = dfs['output'].parallel_apply(replace_bin)

In [180]:
dfs.describe()

,target_num,output_num
count,40000.000000,40000.000000
mean,0.500000,0.634400
std,0.500006,0.481604
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.500000,1.000000
75%,1.000000,1.000000
max,1.000000,1.000000


In [181]:
f1_score(dfs['target_num'], dfs['output_num'])

0.7659114950634698

In [182]:
def assign_top(df,step=200):
    for i in range(0, int(df.shape[0]/step)):
        df.loc[i*step:(i+1)*step-1,'output_num_avg'] = int(round(df.loc[i*step:(i+1)*step-1,'output_num'].mean()))
    return df

In [183]:
dfs = assign_top(dfs)

In [184]:
dfs.describe()

,target_num,output_num,output_num_avg
count,40000.000000,40000.000000,40000.000000
mean,0.500000,0.634400,0.650000
std,0.500006,0.481604,0.476976
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.500000,1.000000,1.000000
75%,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000


In [185]:
f1_score(dfs['target_num'], dfs['output_num_avg'])

0.8434782608695651

In [ ]:
#let's ch